# Graph Genertaion 

In [1]:
import time
import random
import pandas as pd 

# Graph Generation

In [2]:
class Node(object):
    def __init__(self,v,w):
        self.vertex=v
        self.weight=w
        
def calculate_edges(num_vertices, avg):
    return (avg*num_vertices)//2
        
def graph_generation(num_vertices, avg):
    '''
    number of vertices is 5000 numbered from 1 to 4999
    edge weights vary from 1 to 1000
    adjacency list for every vertex is list of 'Node' objects (contains vertex name and weight of the edge)
    '''
    num_edges=0
    map_adj_list=collections.defaultdict(int)
    
    #creating a cycle, connecting the whole graph
    adj_list = [[] for _ in range(num_vertices)]
    for i in range(1,num_vertices):
        edge_weight = random.randint(1,1000)
        adj_list[i].append(Node(i-1,edge_weight))
        adj_list[i-1].append(Node(i,edge_weight))
        map_adj_list[(i-1,i)]=1
        num_edges+=1
    edge_weight = random.randint(1,1000)    
    adj_list[num_vertices-1].append(Node(0,edge_weight)) 
    adj_list[0].append(Node(num_vertices-1,edge_weight))
    map_adj_list[(0,num_vertices-1)]=1
    num_edges+=1
    print("Graph is connected with a cycle")
    
    #call the average number of neighbors calculating function
    avg = calculate_edges(num_vertices, avg)
    
    while num_edges<avg:
        edge_weight = random.randint(1,1000)
        vertex_1 = random.randint(0,num_vertices-1)
        vertex_2 = random.randint(0,num_vertices-1)
        if vertex_1 != vertex_2 and map_adj_list[tuple(sorted([vertex_1,vertex_2]))]!=1:
            adj_list[vertex_1].append(Node(vertex_2,edge_weight))
            adj_list[vertex_2].append(Node(vertex_1,edge_weight))
            map_adj_list[tuple(sorted([vertex_1,vertex_2]))]=1
            num_edges+=1
            
    print("Graph Generation Completed")        
    return adj_list          

# Dijikstra's for MBP Without Heap

In [3]:
class dijikstra_noHeap(object):
    def __init__(self,x):
        self.adj_list = x
        

    def print_path(self,s,t,dad):
        
        stack=[]
        while t and t!=s:
            stack.append(t)
            t=dad[t]
            
        path=str(s)+"-->"
        while len(stack)>1:
            path=path+str(stack.pop())+"-->"
        if len(stack)==1:
            path+=str(stack.pop())
        return path    


    def dijikstra_no_heap(self,s,t): 
        t1=time.time()
        status=['unseen']*5000
        dad=['']*5000
        fringe_weight=[-float('Inf')]*5000
        weight=[-float('INF')]*5000
        
        fringe=[]
        status[s]='intree'
        
        for neigh in self.adj_list[s]:
            status[neigh.vertex]='fringe'
            fringe.append(neigh.vertex)
            weight[neigh.vertex]=neigh.weight
            fringe_weight[neigh.vertex]=weight[neigh.vertex]
            dad[neigh.vertex]=s

        while fringe:
            max_weight = fringe_weight[fringe[0]]
            best_fringe = fringe[0]
            for i in range(1,len(fringe_weight)):
                if fringe_weight[i]>max_weight:
                    max_weight = fringe_weight[i]
                    best_fringe = i
            #remove it from fringe array
            fringe.remove(best_fringe)
            #add the precessed vertex into the tree
            status[best_fringe]='intree'
            #remove processed vertex weight from the array    
            fringe_weight[best_fringe]=float('-INF')
            
            #visit neighbors of the best fringe    
            for neigh in self.adj_list[best_fringe]:
                if status[neigh.vertex] == 'unseen':
                    status[neigh.vertex]='fringe'
                    fringe.append(neigh.vertex)
                    weight[neigh.vertex]=min(weight[best_fringe],neigh.weight)
                    fringe_weight[neigh.vertex]=weight[neigh.vertex]
                    dad[neigh.vertex]=best_fringe
                    
                elif status[neigh.vertex]=='fringe' and weight[neigh.vertex]<min(weight[best_fringe],neigh.weight):
                    weight[neigh.vertex]=min(weight[best_fringe],neigh.weight)
                    fringe_weight[neigh.vertex]=weight[neigh.vertex]
                    dad[neigh.vertex]=best_fringe
                    
        t1=time.time()-t1
        path=self.print_path(s,t,dad)        

        return path,weight[t],t1        

# Heap Implementation

In [4]:
#implmentation of max heap functions : heappush, heappop, heapmax
#a list to maintain the heap structure using weights of the edges
#a list to maintain the node indices of the nodes in the heap
#a list to mantain the heap indices of the nodes in the heap
#size of the heap before the operation has performed
#item to be inserted
#node to be inserted
#index of the heap to be deleted

class heap(object):
    def __init__(self):
        self.D=[-float('INF')]*5000
        self.H=[]
        self.size=0
        self.node_heap=[None]*5000
        
        
    def percUp(self,i):
        parent_idx = (i-1) // 2
        while  parent_idx >= 0:
            if self.D[self.H[i]] > self.D[self.H[parent_idx]]:
                self.node_heap[self.H[parent_idx]] = i
                self.node_heap[self.H[i]] = parent_idx
                
                tmp = self.H[parent_idx]
                self.H[parent_idx] = self.H[i]
                self.H[i] = tmp
                
            elif self.D[self.H[i]] <= self.D[self.H[parent_idx]]:
                return
                
            i = parent_idx
            parent_idx = (i-1) // 2
            
            
    def percDown(self,i):
        while (i * 2 + 1) < self.size:
                mc = self.minChild(i)
                if self.D[self.H[i]] < self.D[self.H[mc]]:
                    self.node_heap[self.H[i]] = mc
                    self.node_heap[self.H[mc]] = i
                    
                    tmp = self.H[i]
                    self.H[i] = self.H[mc]
                    self.H[mc] = tmp
                else:
                    return
                i = mc

    def minChild(self,i):
        if i * 2 + 2 >= self.size:
            return i * 2 + 1
        else:
            if self.D[self.H[i*2+1]] < self.D[self.H[i*2+2]]:
                return i * 2 + 2
            else:
                return i * 2 + 1        
            
    
    def insert_heap(self, D_dij, item, node):
        ''' 
        H : heap, list that maintains mapping between heap index to node index
        D : values of each vertex
        node_heap : list that maintains mapping between node index to heap index
        item : weight of the node to be inserted
        node : node to be inserted
        size : size of the heap before insertion
        '''
        
        self.H.append(node)
        self.D[node]=item
        self.node_heap[node]=self.size
        
        self.size+=1
        
        self.percUp(self.size-1)
        
        return self.H, self.D
    


    def delete_heap(self, D_dij, v):
        ''' 
        H : heap, list that maintains mapping between heap index to node index
        D : values of each vertex
        node_heap : list that maintains mapping between node index to heap index
        ind : index to be deleted
        size : size of the heap before insertion
        '''
        
        self.D=D_dij
        
        ind=self.node_heap[v]
        last_index=self.size-1
        
        #checking if the vertex to be deleted is the last item in the heap
        if ind==last_index:
            self.node_heap[self.H[last_index]]=None
            self.D[self.H[last_index]]=float('-inf')
            self.H.pop(last_index)
            self.size-=1
            return self.H, self.D
        
    
        self.node_heap[self.H[last_index]]=ind
        self.H[ind]=self.H[last_index]
        
        self.D[v]=float('-INF')
        self.node_heap[v]=None
        self.H.pop(last_index)
        
        
        self.size-=1
        
        parent=(ind-1)//2
        child_1, child_2 =(2*ind)+1, (2*ind)+2
        
        #heapify up
        if parent>=0 and self.D[self.H[ind]]>self.D[self.H[parent]]:
            self.percUp(ind) 
      
        #heapify down
        else:
            self.percDown(ind)
            
        return self.H, self.D

    def check_old(self):
        for i in range(5000):
            if self.node_heap[i]!=None and self.H[self.node_heap[i]]!=i:
                return False
        return True 
    
    def check(self):
        for i in range(self.size):
            if self.node_heap[self.H[i]] != i:
                return False
        return True
    


    def max_heap(self):
        if self.size>0:
            return self.H[0], self.D[self.H[0]]
        return None

                
        

# Dijikstra's For MBP using Heap

In [5]:
#dijikstra's with Heap
class dijikstra_withHeap(object):
    def __init__(self,dw_x):
        self.dw_adj_list=dw_x
        
    def print_path(self,dw_s,dw_t,dw_dad):
        
        stack=[]
        while dw_t and dw_t!=dw_s:
            stack.append(dw_t)
            dw_t=dw_dad[dw_t]
            
        path=str(dw_s)+"-->"
        while len(stack)>1:
            path=path+str(stack.pop())+"-->"
        if len(stack)==1:
            path+=str(stack.pop())
            
        return path 
    
    
        
    def dijikstra_with_heap(self,s,t):
        t2=time.time()
        h = heap()
        status=['unseen']*5000
        dad=[None]*5000
        n_h = [None]*5000

        weight=[-float('INF')]*5000
        fringe_weight=[-float('INF')]*5000

        fringe=[]

        status[s]='intree'
                       
        for neigh in self.dw_adj_list[s]:
                status[neigh.vertex]='fringe'
                weight[neigh.vertex]=neigh.weight
                dad[neigh.vertex]=s
                fringe, fringe_weight = h.insert_heap(fringe_weight, weight[neigh.vertex], neigh.vertex)
        
        
        while fringe:
            best_fringe, w = h.max_heap()
            fringe, fringe_weight = h.delete_heap(fringe_weight, best_fringe)
            status[best_fringe]='intree'

            for neigh in self.dw_adj_list[best_fringe]:
                    if status[neigh.vertex] == 'unseen':
                        status[neigh.vertex]='fringe'
                        weight[neigh.vertex]=min(w,neigh.weight)
                        dad[neigh.vertex]=best_fringe
                        fringe, fringe_weight = h.insert_heap(fringe_weight, weight[neigh.vertex], neigh.vertex)
                    
                    elif status[neigh.vertex]=='fringe' and weight[neigh.vertex]<min(w,neigh.weight):
                        weight[neigh.vertex]=min(w,neigh.weight)
                        dad[neigh.vertex]=best_fringe
                        fringe, fringe_weight = h.delete_heap(fringe_weight, neigh.vertex)
                        fringe, fringe_weight = h.insert_heap(fringe_weight, weight[neigh.vertex], neigh.vertex)
                                    
        t2=time.time()-t2
        path=self.print_path(s,t,dad)        

        return path,weight[t],t2  

# Kruskal MBP

In [6]:
import collections
class Subset: 
    def __init__(self, parent, rank): 
        self.parent = parent 
        self.rank = rank 
        


class kruskal(object):
    def __init__(self, k_adj_list):
        self.kruskal_edges=[]
        self.kruskal_dict_edges={}
        self.kruskal_subsets = []
        self.kruskal_T=[]
        self.kruskal_adj_list = k_adj_list
        
    
    def heapify(self, arr, n, i): 
        largest = i 
        l = 2 * i + 1      
        r = 2 * i + 2     

        # See if left child of root exists and is 
        # greater than root 
        if l < n and arr[i][2] < arr[l][2]: 
            largest = l 

        # See if right child of root exists and is 
        # greater than root 
        if r < n and arr[largest][2] < arr[r][2]: 
            largest = r 

        # Change root, if needed 
        if largest != i: 
            tmp = arr[i]
            arr[i] = arr[largest]
            arr[largest] = tmp

            # Heapify the root. 
            self.heapify(arr, n, largest) 
  
    # The main function to sort an array of given size 
    def heapSort(self, arr): 
        n = len(arr) 
  
        # Build a maxheap. 
        for i in range(n-1, -1, -1): 
            self.heapify(arr, n, i) 
            
        for i in range(n-1,0,-1):
            tmp=arr[i]
            arr[i]=arr[0]
            arr[0]=tmp
            self.heapify(arr,i,0)
        
        return arr[::-1]
    
    
    def find(self, subsets, node): 
        if subsets[node].parent != node: 
            subsets[node].parent = self.find(subsets, subsets[node].parent) 
        return subsets[node].parent 


    def union(self, subsets, u, v): 

        # Attach smaller rank tree under root  
        # of high rank tree(Union by Rank) 
        if subsets[u].rank > subsets[v].rank: 
            subsets[v].parent = u 
        elif subsets[v].rank > subsets[u].rank: 
            subsets[u].parent = v 

        # If ranks are same, then make one as  
        # root and increment its rank by one 
        else: 
            subsets[v].parent = u 
            subsets[u].rank += 1
    
        
        
    def print_path(self, k_s, k_t, k_path, k_wt):
        for val in self.kruskal_tmp_edges[k_s]:
            if val.vertex==k_t:
                return [k_path+[k_t], min(k_wt, val.weight)]
            elif val.vertex not in k_path:
                x = self.print_path(val.vertex, k_t, k_path+[val.vertex], min(k_wt, val.weight))
                if x is not None:
                    return x
        
            
    def max_spanning_tree(self, k_s, k_t):
        t3=time.time()

        #creating an array containing information of an edge : vertex pairs, weight
        for key in range(len(self.kruskal_adj_list)):
            for edge in self.kruskal_adj_list[key]:
                if key>edge.vertex:
                    v1,v2= edge.vertex, key
                else:
                    v1,v2=key, edge.vertex
                if (v1,v2) not in self.kruskal_dict_edges:
                    self.kruskal_dict_edges[(v1,v2)]=edge.weight
                    self.kruskal_edges.append([v1,v2,edge.weight])

        #heapsort the edges array
        self.kruskal_edges = self.heapSort(self.kruskal_edges)

        for u in range(5000): 
            self.kruskal_subsets.append(Subset(u, 0))

        for i in range(len(self.kruskal_edges)):
            v1,v2,w = self.kruskal_edges[i][0], self.kruskal_edges[i][1], self.kruskal_edges[i][2]
            root_v1, root_v2 = self.find(self.kruskal_subsets,v1), self.find(self.kruskal_subsets,v2)
            #if the verteices are from different sets, then they do not form a cycle and hence add them in our T collection.
            if root_v1!=root_v2:
                self.union(self.kruskal_subsets,root_v1,root_v2)
                
                self.kruskal_T.append(self.kruskal_edges[i])
                
        self.kruskal_tmp_edges=collections.defaultdict(list)
        for edge in self.kruskal_T:
            self.kruskal_tmp_edges[edge[0]].append(Node(edge[1],edge[2]))
            self.kruskal_tmp_edges[edge[1]].append(Node(edge[0],edge[2]))
        
        
        t3=time.time()-t3
        #return self.kruskal_T, self.kruskal_edges
        kruskal_result = self.print_path(k_s, k_t, [k_s] , float('INF'))
        kruskal_result[0] = '-->'.join(str(x) for x in kruskal_result[0])
        return kruskal_result[0], kruskal_result[1], t3
                
                
                

        

# Dummy Graph Generation

In [78]:
# import collections
# d=collections.defaultdict(list)
# d[0].append(Node(1,10))
# d[0].append(Node(3,20))
# d[0].append(Node(5,14))
# d[1].append(Node(2,5))
# d[1].append(Node(0,10))
# d[2].append(Node(1,5))
# d[2].append(Node(3,30))
# d[2].append(Node(4,15))
# d[3].append(Node(0,20))
# d[3].append(Node(2,30))
# d[4].append(Node(2,15))
# d[4].append(Node(5,16))
# d[5].append(Node(4,16))
# d[5].append(Node(0,14))
# adj_list=d
# #creating an instance for dijikstra's without heap
# dij1 = dijikstra_noHeap(adj_list)
# #creating an instance for dijikstra's with heap
# dij2= dijikstra_withHeap(adj_list)

# res1,res2=[],[]
# p1,w1,t1 = dij1.dijikstra_no_heap(0,5)
# p2,w2,t2 = dij2.dijikstra_with_heap(0,5)
# res1.append([0,4,p1,w1])
# res2.append([0,4,p2,w2])
# #print("Test Case ", i+1)
# print("Source :",res1[0][0], "  Destination :", res1[0][1])
# print("MBP of No Heap :", res1[0][3], " MBP of Heap :", res2[0][3])
# print("Path No Heap :",res1[0][2], " Path Heap ", res2[0][2])

Source : 0   Destination : 4
MBP of No Heap : 15  MBP of Heap : 15
Path No Heap : 0-->3-->2-->4-->5  Path Heap  0-->3-->2-->4-->5


# Testing the Algorithms

In [7]:
import time
outer_iteration = 0
df_sparse = pd.DataFrame(columns=['Source', 'Destination', 'Time Dijikstra No Heap', 'Time Dijikstra with Heap', 'Time Kruskal',
                                 'Maximum Bandwidth Dijikstra No Heap', 'Maximum Bandwidth Dijikstra with Heap', 
                                 'Maximum Bandwidth Kruskal', 'Path Dijikstra No Heap', 'Path Dijikstra with Heap',
                                 'Path Kruskal'])
df_dense = pd.DataFrame(columns=['Source', 'Destination', 'Time Dijikstra No Heap', 'Time Dijikstra with Heap', 'Time Kruskal',
                                 'Maximum Bandwidth Dijikstra No Heap', 'Maximum Bandwidth Dijikstra with Heap', 
                                 'Maximum Bandwidth Kruskal', 'Path Dijikstra No Heap', 'Path Dijikstra with Heap',
                                 'Path Kruskal'])
while outer_iteration<5:
    outer_iteration+=1
    #random graph generation: sparse
    adj_list_1 = graph_generation(5000,6)
    #creating an instance for dijikstra's without heap
    dij_wh_1 = dijikstra_noHeap(adj_list_1)
    #creating an instance for dijikstra's with heap
    dij_h_1= dijikstra_withHeap(adj_list_1)
    #creating an instance for kruskal's
    kruskal_1 = kruskal(adj_list_1)

    iteration=0
    res1,res2,res3=[],[],[]

    #generating 5 different pairs of spurce and destination pairs for every random graph
    while iteration<5:
        start,end = random.randint(0,4999),random.randint(0,4999)
        if start!=end:
            iteration+=1
            p1,w1,t1 = dij_wh_1.dijikstra_no_heap(start,end)
            p2,w2,t2 = dij_h_1.dijikstra_with_heap(start,end)
            p3,w3,t3 = kruskal_1.max_spanning_tree(start,end)
            res1.append([start,end,p1,w1,t1])
            res2.append([start,end,p2,w2,t2])
            res3.append([start,end,p3,w3,t3])

    print("Sparse Graph Output ")
    df_sparse = df_sparse.append({'Source' : [res1[i][0] for i in range(5)],
                  'Destination ' : [res1[i][1] for i in range(5)],

                        'Time Dijikstra No Heap':[res1[i][4] for i in range(5)],
                        'Time Dijikstra with Heap':[res2[i][4] for i in range(5)],
                        'Time Kruskal':[res3[i][4] for i in range(5)],

                        'Maximum Bandwidth Dijikstra No Heap':[res1[i][3] for i in range(5)], 
                        'Maximum Bandwidth Dijikstra with Heap':[res2[i][3] for i in range(5)], 
                        'Maximum Bandwidth Kruskal':[res3[i][3] for i in range(5)],

                        'Path Dijikstra No Heap':[res1[i][2] for i in range(5)],
                        'Path Dijikstra with Heap':[res2[i][2] for i in range(5)],
                        'Path Kruskal':[res3[i][2] for i in range(5)]
                        }, ignore_index=True)  


    #random graph generation: dense
    adj_list_2 = graph_generation(5000,1000)
    #creating an instance for dijikstra's without heap
    dij_wh_2 = dijikstra_noHeap(adj_list_2)
    #creating an instance for dijikstra's with heap
    dij_h_2 = dijikstra_withHeap(adj_list_2)
    #creating an instance for kruskal's
    kruskal_2 = kruskal(adj_list_2)

    iteration=0
    res4,res5,res6=[],[],[]

    #generating 5 different pairs of spurce and destination pairs for every random graph
    while iteration<5:
        start,end = random.randint(0,4999),random.randint(0,4999)
        if start!=end:
            iteration+=1
            p4,w4,t4 = dij_wh_2.dijikstra_no_heap(start,end)
            p5,w5,t5 = dij_h_2.dijikstra_with_heap(start,end)
            p6,w6,t6 = kruskal_2.max_spanning_tree(start,end)
            res4.append([start,end,p4,w4,t4])
            res5.append([start,end,p5,w5,t5])
            res6.append([start,end,p6,w6,t6])

    print("Dense Graph Output ")
    df_dense = df_dense.append({'Source' : [res4[i][0] for i in range(5)],
                  'Destination ' : [res4[i][1] for i in range(5)],

                        'Time Dijikstra No Heap':[res4[i][4] for i in range(5)],
                        'Time Dijikstra with Heap':[res5[i][4] for i in range(5)],
                        'Time Kruskal':[res6[i][4] for i in range(5)],

                        'Maximum Bandwidth Dijikstra No Heap':[res4[i][3] for i in range(5)], 
                        'Maximum Bandwidth Dijikstra with Heap':[res5[i][3] for i in range(5)], 
                        'Maximum Bandwidth Kruskal':[res6[i][3] for i in range(5)],

                        'Path Dijikstra No Heap':[res4[i][2] for i in range(5)],
                        'Path Dijikstra with Heap':[res5[i][2] for i in range(5)],
                        'Path Kruskal':[res6[i][2] for i in range(5)]
                        },ignore_index=True)  


Graph is connected with a cycle
Graph Generation Completed
Sparse Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Dense Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Sparse Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Dense Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Sparse Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Dense Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Sparse Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Dense Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Sparse Graph Output 
Graph is connected with a cycle
Graph Generation Completed
Dense Graph Output 


In [8]:
print('Sparse Graph')
print('Dijikstra without heap   Dijikstra using Heaps     Kruskals')
print("")

for i in range(5):
    print(df_sparse['Time Dijikstra No Heap'][0][i], "      ", df_sparse['Time Dijikstra with Heap'][0][i] , "     ", df_sparse['Time Kruskal'][0][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________") 
print("")

for i in range(5):
    print(df_sparse['Time Dijikstra No Heap'][1][i], "      ", df_sparse['Time Dijikstra with Heap'][1][i] , "     ", df_sparse['Time Kruskal'][1][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")
print("")

for i in range(5):
    print(df_sparse['Time Dijikstra No Heap'][2][i], "      ", df_sparse['Time Dijikstra with Heap'][2][i] , "     ", df_sparse['Time Kruskal'][2][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")
print("")

for i in range(5):
    print(df_sparse['Time Dijikstra No Heap'][3][i], "      ", df_sparse['Time Dijikstra with Heap'][3][i] , "     ", df_sparse['Time Kruskal'][3][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")
print("")

for i in range(5):
    print(df_sparse['Time Dijikstra No Heap'][4][i], "      ", df_sparse['Time Dijikstra with Heap'][4][i] , "     ", df_sparse['Time Kruskal'][4][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")

    

Sparse Graph
Dijikstra without heap   Dijikstra using Heaps     Kruskals

1.6641390323638916        0.1386699676513672       0.19594693183898926
-------------------------------------------------------------------------
1.6870179176330566        0.1340630054473877       0.19690394401550293
-------------------------------------------------------------------------
1.6872150897979736        0.14044809341430664       0.16860198974609375
-------------------------------------------------------------------------
1.6246600151062012        0.11615586280822754       0.16472601890563965
-------------------------------------------------------------------------
1.568392038345337        0.1114809513092041       0.18155312538146973
-------------------------------------------------------------------------
______________________________________________________________________________

1.604675054550171        0.13699817657470703       0.17036223411560059
-------------------------------------------------

In [9]:
print('Dense Graph')
print('Dijikstra without heap   Dijikstra using Heaps     Kruskals')
print("")

for i in range(5):
    print(df_dense['Time Dijikstra No Heap'][0][i], "      ", df_dense['Time Dijikstra with Heap'][0][i] , "     ", df_dense['Time Kruskal'][0][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________") 
print("")

for i in range(5):
    print(df_dense['Time Dijikstra No Heap'][1][i], "      ", df_dense['Time Dijikstra with Heap'][1][i] , "     ", df_dense['Time Kruskal'][1][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")
print("")

for i in range(5):
    print(df_dense['Time Dijikstra No Heap'][2][i], "      ", df_dense['Time Dijikstra with Heap'][2][i] , "     ", df_dense['Time Kruskal'][2][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")
print("")

for i in range(5):
    print(df_dense['Time Dijikstra No Heap'][3][i], "      ", df_dense['Time Dijikstra with Heap'][3][i] , "     ", df_dense['Time Kruskal'][3][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")
print("")

for i in range(5):
    print(df_dense['Time Dijikstra No Heap'][4][i], "      ", df_dense['Time Dijikstra with Heap'][4][i] , "     ", df_dense['Time Kruskal'][4][i])
    print("-------------------------------------------------------------------------")
print("______________________________________________________________________________")

    

Dense Graph
Dijikstra without heap   Dijikstra using Heaps     Kruskals

6.645751714706421        5.052077770233154       62.03793382644653
-------------------------------------------------------------------------
6.301461935043335        4.704345226287842       45.00173592567444
-------------------------------------------------------------------------
7.433055877685547        5.1156089305877686       45.41713500022888
-------------------------------------------------------------------------
7.061755180358887        5.186711072921753       44.67284083366394
-------------------------------------------------------------------------
6.775209903717041        4.988193035125732       44.743908166885376
-------------------------------------------------------------------------
______________________________________________________________________________

6.756299734115601        5.0365025997161865       62.83172392845154
------------------------------------------------------------------------